In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型监控用于AutoML表格模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> 在Colab中运行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fget_started_with_model_monitoring_automl.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png\" alt="Google Cloud Colab Enterprise logo\"><br> 在Colab Enterprise中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      在GitHub上查看
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_automl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

概述

本教程演示了如何使用Vertex AI Model Monitoring监控AutoML表格模型。

了解有关[Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring)的更多信息。

### 目标

在这个笔记本中，您将学习如何使用Vertex AI模型监控服务来检测AutoML表格模型中的特征偏差和漂移。

本教程使用以下Vertex AI服务：

- AutoML
- Vertex AI模型监控
- Vertex AI预测
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 训练一个AutoML模型。
- 将模型资源部署到Vertex AI端点资源。
- 配置端点资源进行模型监控。
- 生成用于特征偏差的合成预测请求。
- 生成用于漂移的合成预测请求。
- 等待电子邮件警报通知。

了解更多关于[介绍Vertex AI模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。在本笔记本中，您仅使用数据集中的年份、月份和日期字段来预测每日平均温度（mean_temp）的值。

### 成本

本教程使用了谷歌云的可计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)基于您预计的使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
# Install required packages.
! pip3 install  --quiet --upgrade google-cloud-aiplatform \
                                  google-cloud-bigquery

### 重新启动运行时（仅适用于Colab）

为了使用新安装的软件包，您必须重新启动Google Colab的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。请等待完成后再继续下一步。⚠️</b>
</div>

验证您在Google Colab上的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息

请阅读有关[设置项目和开发环境的更多信息](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"

### 用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

关于服务账号和权限的说明

**默认情况下不需要配置**，如果遇到任何权限相关问题，请确保上述服务账号具有所需的角色：

|服务账号邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine 默认服务账号|Dataflow Admin、Dataflow Worker、Storage Admin、BigQuery Admin、Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform 服务代理|Vertex AI 服务代理|

1. 进入[IAM控制台](https://console.cloud.google.com/iam-admin/iam)。
2. 选中**包括由谷歌提供的角色授予**复选框。
3. 找到上述邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
如果您正在使用来自不同项目的数据源：
- 对于 BigQuery 数据源，请将"BigQuery 数据查看器"角色授予这两个服务账号。
- 对于 CSV 数据源，请将"存储对象查看器"角色授予这两个服务账号。

导入库

In [ ]:
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须 [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 创建BigQuery客户端

在本教程中，您将使用来自公共BigQuery表的数据。您将创建一个客户端接口，随后用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

##介绍Vertex AI模型监控

Vertex AI模型监控支持AutoML表格模型和自定义表格模型。您可以监控传入预测请求中的特征的倾斜和漂移检测，或者监控传出预测响应中的特征归因（可解释AI）。换句话说，您监控量化特征对输出（预测）的贡献的归因分布。

以下是启用模型监控的基本步骤：

1. 将Vertex AI AutoML或自定义表格模型部署到Vertex AI端点。
2. 配置模型监控规范。
3. 上传模型监控规范到Vertex AI端点。
4. 上传模式或使用*输入模式*的自动生成进行解析。
5. 对于特征倾斜检测，上传训练数据。这使得自动生成特征分布。
6. 对于特征归因，上传相应的*Vertex可解释AI*规范。

配置完成后，您可以启用或禁用监控，更改警报并更新模型监控配置。

启用模型监控后，抽样的传入预测请求将被记录到一个BigQuery表中。然后，分析记录请求中的输入特征值，以检测特定时间间隔的倾斜或漂移。您设置一个采样率，以监控模型的一部分生产输入，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于AutoML表格模型，输入模式会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动派生输入模式。或者，您可以上传输入模式。

对于倾斜检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格模型，这是自动生成的。对于自定义表格模型，您需要将训练数据上传到服务中，并让服务自动派生分布。

对于特征归因倾斜和特征归因中的漂移检测，您需要为部署的自定义表格模型启用Vertex可解释AI功能。对于AutoML模型，Vertex可解释AI会自动启用。

了解更多关于[Vertex AI模型监控介绍](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

#### BigQuery培训数据的位置。

现在将变量`IMPORT_FILE`设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 创建数据集

#### BigQuery 输入数据

接下来，使用 `create` 方法为 `TabularDataset` 类创建数据集资源，该方法接受以下参数：

- `display_name`：数据集资源的人类可读名称。
- `bq_source`：将数据项从 BigQuery 表导入到数据集资源中。
- `labels`：用户定义的元数据。在这个示例中，您将存储包含用户定义数据的 Cloud Storage 存储桶的位置。

了解更多关于[从 BigQuery 表创建 TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)的信息。

In [ ]:
dataset = aiplatform.TabularDataset.create(display_name="gsod", bq_source=[IMPORT_FILE])

label_column = "mean_temp"

print(dataset.resource_name)

### 创建并运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流水线，2) 运行该流水线。

#### 创建训练流水线

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练流水线，包括以下参数：

- `display_name`：训练作业资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：表格分类模型。
  - `regression`：表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转化。在这个例子中，您将设置列转换以使用基于数据类型的默认转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

#### 运行训练流程

接下来，您可以运行创建的DAG来启动训练作业，通过调用`run`方法，并将以下参数传入：

- `dataset`：用于训练模型的数据集资源。
- `model_display_name`：已训练模型的易读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000=小时）。
- `disable_early_stopping`：如果为`False`，则在服务认为无法进一步改进模型目标测量时，训练可能在使用整个预算之前完成。

完成`run`方法后，将返回模型资源。

训练流程的执行可能需要最多30分钟甚至更长时间。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您可以调用 `deploy` 方法，并使用以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

### 配置警报规范

首先，您配置`alerting_config`规范，其设置如下：

- `user_emails`：要发送警报的一个或多个电子邮件列表。
- `enable_logging`：将检测到的异常流式传输到云日志。默认值为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监视间隔规范

接下来，您将使用以下设置配置`schedule_config`规范：

- `monitor_interval`： 设置模型监视作业调度间隔时间（以小时为单位）。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置抽样规范

接下来，您将使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`：将预测请求随机抽样用于监控的速率（介于0和1之间的百分比）。选定的样本将被记录到一个BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您将使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对字典，其中键是要监视漂移的输入特征，值是检测阈值。 当未指定时，默认特征的漂移阈值为0.3（30％）。

**注意：** 启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {"year": DRIFT_THRESHOLD_VALUE, "motnth": DRIFT_THRESHOLD_VALUE}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置偏斜检测规范

接下来，您可以使用以下设置配置`skew_config`规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为BigQuery表。否则，必须将设置`data_format`设置为以下值之一。数据的位置必须是Cloud Storage位置。
  - `csv`
  - `jsonl`
  - `tf-record`
- `skew_thresholds`：一个键/值对字典，其中键是要监视偏斜的输入特征。值是检测阈值。如果未指定，特征的默认偏斜阈值为0.3（30%）。
- `target_field`：训练数据集的目标标签

**注意：**启用偏斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://" + BQ_TABLE
# Prediction target column name in training dataset.
TARGET = label_column

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "year": SKEW_THRESHOLD_VALUE,
    "month": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装客观规格

最后，您将使用以下设置组装客观规格（`objective_config`）：

- `skew_detection_config`：（可选）用于倾斜检测配置的规格。
- `drift_detection_config`：（可选）用于漂移检测配置的规格。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型，这被称为*输入模式*。

对于AutoML模型，*输入模式*是预定义的，并由监控服务自动加载。

### 创建监控任务

您可以使用 `aiplatform.ModelDeploymentMonitoringJob.create()` 方法创建一个监控任务，根据您的监控规格，可以设置以下参数：

- `display_name`: 监控任务的人类可读名称。
- `project`: 项目ID。
- `location`: 地点。
- `endpoint`: 启用监控的 Vertex AI 端点的完全限定资源名称。
- `logging_sampling_strategy`: 采样配置规范。
- `schedule_config`: 调度配置规范。
- `alert_config`: 告警配置规范。
- `objective_configs`: 目标配置规范。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=LOCATION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job)

#### 监控任务的电子邮件通知。

将电子邮件通知发送到警报配置中的电子邮件地址，通知模型监控任务现在已启用。

电子邮件的内容如下：

<blockquote>
您好 Vertex AI 客户，

您收到此邮件是因为您正在使用 Vertex AI 模型监控服务。
此邮件是通知您，我们已收到您设置漂移或偏差检测的请求，以用于下面列出的预测端点。从现在开始，传入的预测请求将被抽样并记录以供分析。
原始请求和响应将从预测服务中收集，并保存在 bq://[您的项目ID].model_deployment_monitoring_[端点ID].serving_predict 中。
</blockquote>

*备注：* 您无需等待电子邮件通知即可继续下一步。

### 监控作业状态

在启动 Vertex AI 模型监控作业之后，它会处于 **PENDING** 状态，直到`偏斜分布基线`被计算出来。监控服务会启动一个批处理作业，从训练数据中生成分布基线。

一旦基线分布生成，监控作业就会变为 **OFFLINE** 状态。按照间隔，比如每小时一次，监控作业会进入 **RUNNING** 状态，分析采样数据。一旦分析完成，它会回到 **OFFLINE** 状态，准备下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter="display_name=churn")
job = jobs[0]
print(job.state)

自动生成基准分布

接下来，监控服务创建批处理作业来分析训练数据以生成基准分布。 完成后，监控服务开始在指定的间隔上进行监视。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(180)

### 为偏斜检测生成合成预测请求

现在，您从BigQuery训练表中提取前1000个实例，用于创建预测请求。然后，通过进行以下更新修改数据（合成数据），以触发预测请求中的偏斜检测：

- `year`：将所有值设定为3。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "year":
            value = "3"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您可以使用`predict()`方法向您的Vertex AI端点资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录采样请求

一旦监控服务启动，采样的预测请求将被记录到云存储中。在下一个监控间隔期间，采样的预测将被复制到BigQuery的日志表中。一旦条目被记录，监控服务会分析采样数据。

接下来，您等待第一个被记录的条目出现在BigQuery的预测样本日志表中。由于您发送了1000个预测请求，采样率为50％，您应该看到大约500条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 监控期间的偏斜检测

下一个监控间隔将进行特征输入的偏斜检测。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业从**离线**变为**运行**。在运行时，它会分析此间隔中的预测日志表，并将其与基线分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的偏斜，本例中为 `year`。此后，监控作业进入**离线**状态，直到下一个间隔。

#### 等待监控间隔

从分析监控间隔完成到接收电子邮件警报可能需要40分钟或更长时间。

电子邮件的内容如下：

<blockquote>
  您好，Vertex AI 客户，

您收到此邮件是因为您订阅了Vertex AI模型监控服务。
此邮件仅用于通知您检测到您部署模型中的一些异常情况，可能需要您的关注。

基本信息：

终端名称：projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监控作业：projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计信息和异常根路径（Google Cloud Storage）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery 命令：SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`

训练预测偏斜异常（原始特征）：

异常报告路径（Google Cloud Storage）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关警报的更多信息，请访问模型监控警报页面。

部署的模型 id：<deployed-model-id>

特征名称	异常短描述	异常长描述
country	训练和服务之间的高Linfty距离	训练和服务之间的Linfty距离为0.947563（最多六个有效数字），高于阈值0.5。具有最大差异的特征值为：Year
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

为漂移检测生成合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例，用于预测请求。然后，修改数据（合成数据），触发预测请求中的漂移检测，方法是进行以下更新：

- `月份`：将所有值设为1。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "month":
            value = "1"
        instance[key] = str(value)
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您可以使用`predict()`方法向您的Vertex AI端点资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在下一个监控间隔中，抽样预测结果会被复制到BigQuery的记录表中。一旦记录完成，监控服务会分析抽样数据。

接下来，您需要等待第一个被记录的条目出现在BigQuery的记录表中以获取预测抽样数据。由于您发送了1000个预测请求，抽样率为50%，您应该会看到大约1000条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 505:
        break

### 在监控过程中的漂移检测

特征输入漂移检测发生在下一个监控间隔期间。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业从**离线**状态变为**运行**状态。运行时，它分析在此间隔期间预测的已记录样本表，并将它们与上一个监控间隔期间的分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的漂移，此处为 `cnt_user_engagement`，并且监控作业进入**离线**状态，直到下一个间隔期间。

#### 等待监控间隔

从监控间隔的分析完成时起，直到收到电子邮件提醒，可能需要40分钟或更长时间。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 删除监控任务

一旦您收到了邮件警报并验证了内容，您可以：
- 使用`pause()`方法暂停监控任务。
- 使用`delete()`方法删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

清理工作

要清理此项目中使用的所有谷歌云资源，您可以[删除您用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Undeploy the model from endpoint before deletion
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the BigQuery table
! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}